<a href="https://colab.research.google.com/github/ad71/ragbot/blob/master/langgraph_persistence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%pip install -qU langchain langchain_openai tavily-python langchain_community langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 918.7 kB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# optional, add tracing in LangSmith
os.environ['LANGCHAIN_TRACING_V2'] = 'true'

In [4]:
# setup tools
from langchain_community.tools.tavily_search import TavilySearchResults
tools = [TavilySearchResults(max_results=1)]

In [6]:
from langgraph.prebuilt import ToolExecutor
tool_executor = ToolExecutor(tools)

In [7]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(temperature=0, streaming=True)

In [8]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

In [9]:
from langgraph.prebuilt import ToolInvocation
from langchain_core.messages import FunctionMessage
import json

def should_continue(messages):
    last_message = messages[-1]

    if 'function_call' not in last_message.additional_kwargs:
        return 'end'
    else:
        return 'continue'

def call_model(messages):
    response = model.invoke(messages)
    return response

def call_tool(messages):
    last_message = messages[-1]
    action = ToolInvocation(
        tool=last_message.additional_kwargs['function_call']['name'],
        tool_input=json.loads(last_message.additional_kwargs['function_call']['arguments'])
    )
    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)
    return function_message

In [10]:
from langgraph.graph import MessageGraph, END
workflow = MessageGraph()

workflow.add_node('agent', call_model)
workflow.add_node('action', call_tool)

workflow.set_entry_point('agent')

workflow.add_conditional_edges(
    'agent',
    should_continue,
    {
        'continue': 'action',
        'end': END
    }
)

workflow.add_edge('action', 'agent')

### Persistence

In [11]:
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(':memory:')

In [12]:
app = workflow.compile(checkpointer=memory)

In [13]:
from langchain_core.messages import HumanMessage
inputs = [HumanMessage(content="Hi! I'm bob")]
for event in app.stream(inputs, {'configurable': {'thread_id': '2'}}):
    for k, v in event.items():
        if k != '__end__':
            print(v)

content='Hello Bob! How can I assist you today?' response_metadata={'finish_reason': 'stop'} id='run-003fb847-e361-4ac9-8731-0956ffe11a0d-0'


In [14]:
inputs = [HumanMessage(content='what is my name?')]
for event in app.stream(inputs, {'configurable': {'thread_id': '2'}}):
    for k, v in event.items():
        if k != '__end__':
            print(v)

content='Your name is Bob! How can I help you, Bob?' response_metadata={'finish_reason': 'stop'} id='run-86e302c9-e6fb-4444-bd36-3f358e6d8c62-0'


In [15]:
inputs = [HumanMessage(content='what is my name?')]
for event in app.stream(inputs, {'configurable': {'thread_id': '3'}}):
    for k, v in event.items():
        if k != '__end__':
            print(v)

content="I'm sorry, but I don't have access to your personal information such as your name. How can I assist you today?" response_metadata={'finish_reason': 'stop'} id='run-477675ee-f04f-45ae-8fb9-3ebffde6b911-0'
